In [1]:
import tensorflow as tf 
import numpy as np 
from tensorflow import keras 
import os 
import math 
import random 
import pickle 
import glob
import matplotlib.pyplot as plt 
from collections import deque 
from tensorflow.keras import layers
import time 

from vehicle_model_DDPG1 import Environment 
from cell_model import CellModel 

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
drving_cycle = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
battery_path = "../../OC_SIM_DB/OC_SIM_DB_Bat/OC_SIM_DB_Bat_nimh_6_240_panasonic_MY01_Prius.mat"
motor_path = "../../OC_SIM_DB/OC_SIM_DB_Mot/OC_SIM_DB_Mot_pm_95_145_X2.mat"
cell_model = CellModel()
env = Environment(cell_model, drving_cycle, battery_path, motor_path, 10)

num_states = 4

In [3]:
class OUActionNoise: 
    def __init__(self, mean, std_deviation, theta=0.15, dt=1e-2, x_initial=None): 
        self.theta = theta 
        self.mean = mean 
        self.std_dev = std_deviation 
        self.dt = dt 
        self.x_initial = x_initial 
        self.reset() 
        
    def reset(self): 
        if self.x_initial is not None: 
            self.x_prev = self.x_initial 
        else: 
            self.x_prev = 0 
            
    def __call__(self): 
        x = (
             self.x_prev + self.theta * (self.mean - self.x_prev) * self.dt 
            + self.std_dev * np.sqrt(self.dt) * np.random.normal() 
        )
        self.x_prev = x 
        return x 

In [4]:
class Buffer: 
    def __init__(self, buffer_capacity=100000, batch_size=64):      
        self.buffer_capacity = buffer_capacity 
        self.batch_size = batch_size 
        self.buffer_counter = 0 
        
        self.state_buffer = np.zeros((self.buffer_capacity, num_states))
        self.action_buffer = np.zeros((self.buffer_capacity, 1))
        self.reward_buffer = np.zeros((self.buffer_capacity, 1))
        self.next_state_buffer = np.zeros((self.buffer_capacity, num_states))
        
    def record(self, obs_tuple):
        index = self.buffer_counter % self.buffer_capacity 

        self.state_buffer[index] = obs_tuple[0]
        self.action_buffer[index] = obs_tuple[1]
        self.reward_buffer[index] = obs_tuple[2]
        self.next_state_buffer[index] = obs_tuple[3]
        
        self.buffer_counter += 1 
        
    def learn(self): 
        record_range = min(self.buffer_counter, self.buffer_capacity)
        batch_indices = np.random.choice(record_range, self.batch_size)

        state_batch = tf.convert_to_tensor(self.state_buffer[batch_indices])
        action_batch = tf.convert_to_tensor(self.action_buffer[batch_indices])
        reward_batch = tf.convert_to_tensor(self.reward_buffer[batch_indices])
        reward_batch = tf.cast(reward_batch, dtype=tf.float32)
        next_state_batch = tf.convert_to_tensor(self.next_state_buffer[batch_indices])
        
        with tf.GradientTape() as tape: 
            target_actions = target_actor(next_state_batch)
            y = reward_batch + gamma * target_critic([next_state_batch, target_actions])
            critic_value = critic_model([state_batch, action_batch])
            critic_loss = tf.math.reduce_mean(tf.square(y - critic_value)) 
        critic_grad = tape.gradient(critic_loss, critic_model.trainable_variables) 
        critic_optimizer.apply_gradients(
            zip(critic_grad, critic_model.trainable_variables)
        )
        
        with tf.GradientTape() as tape: 
            actions = actor_model(state_batch)
            critic_value = critic_model([state_batch, actions])
            actor_loss = - tf.math.reduce_mean(critic_value)
        actor_grad = tape.gradient(actor_loss, actor_model.trainable_variables) 
        actor_optimizer.apply_gradients(
            zip(actor_grad, actor_model.trainable_variables)
        )
        

In [5]:
def update_target(tau): 
    new_weights = [] 
    target_variables = target_critic.weights
    for i, variable in enumerate(critic_model.weights): 
        new_weights.append(target_variables[i] * (1 - tau) + tau * variable)
    target_critic.set_weights(new_weights)
    
    new_weights = [] 
    target_variables = target_actor.weights
    for i, variable in enumerate(actor_model.weights): 
        new_weights.append(target_variables[i] * (1 - tau) + tau * variable)
    target_actor.set_weights(new_weights)
    

In [6]:
def get_actor(): 
    last_init = tf.random_uniform_initializer(minval=-0.003, maxval=0.003)
    
    inputs = layers.Input(shape=(num_states))
    inputs_batchnorm = layers.BatchNormalization()(inputs)
    
    out = layers.Dense(512, activation="relu")(inputs_batchnorm)
#     out = layers.BatchNormalization()(out)
    out = layers.Dense(512, activation="relu")(out)
#     out = layers.BatchNormalization()(out)
    outputs = layers.Dense(1, activation="sigmoid", 
                          kernel_initializer=last_init)(out)
    model = tf.keras.Model(inputs, outputs)
    return model

In [7]:
def get_critic(): 
    state_input = layers.Input(shape=(num_states))
    state_input_batchnorm = layers.BatchNormalization()(state_input)
    
    state_out = layers.Dense(16, activation="relu")(state_input_batchnorm)
#     state_out = layers.BatchNormalization()(state_out)
    state_out = layers.Dense(32, activation="relu")(state_out)
#     state_out = layers.BatchNormalization()(state_out)
    
    action_input = layers.Input(shape=(1))
    action_out = layers.Dense(32, activation="relu")(action_input)
#     action_out = layers.BatchNormalization()(action_out)
    
    concat = layers.Concatenate()([state_out, action_out]) 
    
    out = layers.Dense(512, activation="relu")(concat)
#     out = layers.BatchNormalization()(out)
    out = layers.Dense(512, activation="relu")(out)
#     out = layers.BatchNormalization()(out)
    outputs = layers.Dense(1)(out)
    
    model = tf.keras.Model([state_input, action_input], outputs)
    return model 
    

In [8]:
def policy(state, noise_object): 
    j_min = state[0][2].numpy()
    j_max = state[0][3].numpy()
    sampled_action = tf.squeeze(actor_model(state)) 
    noise = noise_object()
    sampled_action = sampled_action.numpy() + noise 
    legal_action = sampled_action * j_max 
    legal_action = np.clip(legal_action, j_min, j_max)
#     print(j_min, j_max, legal_action, noise)
    return legal_action 
    

In [9]:
def policy_epsilon_greedy(state, eps): 
    j_min = state[0][-2].numpy()
    j_max = state[0][-1].numpy()

    if random.random() < eps: 
        a = random.randint(0, 9)
        return np.linspace(j_min, j_max, 10)[a]
    else: 
        sampled_action = tf.squeeze(actor_model(state)).numpy()  
        legal_action = sampled_action * j_max 
        legal_action = np.clip(legal_action, j_min, j_max)
        return legal_action

In [10]:
std_dev = 0.2 
ou_noise = OUActionNoise(mean=0, std_deviation=0.2)

critic_lr = 0.0005 
actor_lr = 0.00025 
critic_optimizer = tf.keras.optimizers.Adam(critic_lr)
actor_optimizer = tf.keras.optimizers.Adam(actor_lr)

total_episodes = 500
gamma = 0.95 
tau = 0.001 

MAX_EPSILON = 1 
MIN_EPSILON = 0.01 
DECAY_RATE = 0.00002
BATCH_SIZE = 32 
DELAY_TRAINING = 10000 

In [11]:
def initialization(): 
    actor_model = get_actor() 
    critic_model = get_critic() 

    target_actor = get_actor() 
    target_critic = get_critic() 
    target_actor.set_weights(actor_model.get_weights())
    target_critic.set_weights(critic_model.get_weights())
    
    buffer = Buffer(500000, BATCH_SIZE)
    return actor_model, critic_model, target_actor, target_critic, buffer

In [12]:
def save_weights(actor_model, critic_model, target_actor, target_critic, root): 
    actor_model.save_weights("./{}/actor_model_checkpoint".format(root))
    critic_model.save_weights("./{}/critic_model_checkpoint".format(root))
    target_actor.save_weights("./{}/target_actor_checkpoint".format(root))
    target_critic.save_weights("./{}/target_critic_checkpoint".format(root))
    print("model is saved..")

In [13]:
def initialization_env(driving_path, reward_factor):
    env = Environment(cell_model, driving_path, battery_path, motor_path, reward_factor)
    return env 

In [14]:
print(env.version)

num_trials = 3
results_dict = {} 
driving_cycle_paths = glob.glob("training/*.mat")[:1]

for trial in range(num_trials): 
    print("")
    print("Trial {}".format(trial))
    print("")
    
    actor_model, critic_model, target_actor, target_critic, buffer = initialization()
    
    eps = MAX_EPSILON 
    steps = 0
    
    episode_rewards = [] 
    episode_SOCs = [] 
    episode_FCs = [] 
    for ep in range(total_episodes): 
        driving_cycle_path = np.random.choice(driving_cycle_paths)
        print(driving_cycle_path)
        env = initialization_env(driving_cycle_path, 10)
        
        start = time.time() 
        state = env.reset() 
        episodic_reward = 0 

        while True: 
            tf_state = tf.expand_dims(tf.convert_to_tensor(state), 0)
            action = policy_epsilon_greedy(tf_state, eps)
    #         print(action)
            next_state, reward, done = env.step(action)
            if done: 
                next_state = [0] * num_states 

            buffer.record((state, action, reward, next_state))
            episodic_reward += reward 

            if steps > DELAY_TRAINING: 
                buffer.learn() 
                update_target(tau)
                eps = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * np.exp(-DECAY_RATE * (steps
                                                                        -DELAY_TRAINING))

            steps += 1

            if done: 
                break 

            state = next_state 

        elapsed_time = time.time() - start 
        print("elapsed_time: {:.3f}".format(elapsed_time))
        episode_rewards.append(episodic_reward) 
        episode_SOCs.append(env.SOC)
        episode_FCs.append(env.fuel_consumption) 

    #     print("Episode * {} * Avg Reward is ==> {}".format(ep, avg_reward))
        SOC_deviation_history = np.sum(np.abs(np.array(env.history["SOC"]) - 0.6)) 
        print(
              'Episode: {}'.format(ep + 1),
              "Exploration P: {:.4f}".format(eps),
              'Total reward: {}'.format(episodic_reward), 
              "SOC: {:.4f}".format(env.SOC), 
              "Cumulative_SOC_deviation: {:.4f}".format(SOC_deviation_history), 
              "Fuel Consumption: {:.4f}".format(env.fuel_consumption), 
        )
        print("")
    
    root = "DDPG1_trial{}".format(trial+1)
    save_weights(actor_model, critic_model, target_actor, target_critic, root)
    
    results_dict[trial + 1] = {
        "rewards": episode_rewards, 
        "SOCs": episode_SOCs, 
        "FCs": episode_FCs 
    }

1

Trial 0

training\03_nedc.mat


C:\Users\ValueCreatorSong\Desktop\Academic\graduate_paper\degradation_model\experiment\generalization3\vehicle_model_DDPG1.py:251: RuntimeWarning: invalid value encountered in double_scalars
  del_i = (1 / (2 * r_cha)) * (v_cha - (v_cha ** 2 - 4 * r_cha * p_bat) ** (0.5)) * (p_bat < 0) + (1 / (
C:\Users\ValueCreatorSong\Desktop\Academic\graduate_paper\degradation_model\experiment\generalization3\vehicle_model_DDPG1.py:277: RuntimeWarning: invalid value encountered in double_scalars
  del_i = (1 / (2 * r_cha)) * (v_cha - (v_cha ** 2 - 4 * r_cha * p_bat) ** (0.5)) * (p_bat < 0) + (1 / (


SOC is nan...
elapsed_time: 14.109
Episode: 1 Exploration P: 1.0000 Total reward: -3076.0200908294455 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 48.5147

training\03_nedc.mat
SOC is nan...
elapsed_time: 14.236
Episode: 2 Exploration P: 1.0000 Total reward: -3173.4850215001075 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 49.9365

training\03_nedc.mat


C:\Users\ValueCreatorSong\Desktop\Academic\graduate_paper\degradation_model\experiment\generalization3\vehicle_model_DDPG1.py:252: RuntimeWarning: invalid value encountered in double_scalars
  2 * r_dis)) * (v_dis - (v_dis ** 2 - 4 * r_dis * p_bat) ** (0.5)) * (p_bat >= 0)
C:\Users\ValueCreatorSong\Desktop\Academic\graduate_paper\degradation_model\experiment\generalization3\vehicle_model_DDPG1.py:278: RuntimeWarning: invalid value encountered in double_scalars
  2 * r_dis)) * (v_dis - (v_dis ** 2 - 4 * r_dis * p_bat) ** (0.5)) * (p_bat >= 0)


SOC is nan...
elapsed_time: 14.005
Episode: 3 Exploration P: 1.0000 Total reward: -3222.044061353341 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 50.2340

training\03_nedc.mat
SOC is nan...
elapsed_time: 16.707
Episode: 4 Exploration P: 1.0000 Total reward: -3149.5395157929743 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 49.6595

training\03_nedc.mat
SOC is nan...
elapsed_time: 13.819
Episode: 5 Exploration P: 1.0000 Total reward: -3057.7346621001543 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 49.1951

training\03_nedc.mat
SOC is nan...
elapsed_time: 13.820
Episode: 6 Exploration P: 1.0000 Total reward: -3122.0378631306066 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 49.4498

training\03_nedc.mat
SOC is nan...
elapsed_time: 13.764
Episode: 7 Exploration P: 1.0000 Total reward: -3038.5932982566155 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 48.4302

training\03_nedc.mat
SOC is nan...
elapsed_time: 12.890
Episode: 8 Exploratio

...

SOC is nan...
elapsed_time: 68.835
Episode: 141 Exploration P: 0.0558 Total reward: -2082.540763569654 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 3.8407

training\03_nedc.mat
SOC is nan...
elapsed_time: 69.659
Episode: 142 Exploration P: 0.0548 Total reward: -2076.929332384878 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 3.5358

training\03_nedc.mat
SOC is nan...
elapsed_time: 69.252
Episode: 143 Exploration P: 0.0539 Total reward: -2027.2139755633716 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 3.7551

training\03_nedc.mat
SOC is nan...
elapsed_time: 68.956
Episode: 144 Exploration P: 0.0530 Total reward: -2035.1982914981866 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 4.0729

training\03_nedc.mat
SOC is nan...
elapsed_time: 69.236
Episode: 145 Exploration P: 0.0520 Total reward: -2058.2167217786387 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 3.7166

training\03_nedc.mat
SOC is nan...
elapsed_time: 68.680
Episode: 146 Expl

SOC is nan...
elapsed_time: 65.376
Episode: 185 Exploration P: 0.0280 Total reward: -2128.355631304398 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 2.6371

training\03_nedc.mat
SOC is nan...
elapsed_time: 68.699
Episode: 186 Exploration P: 0.0276 Total reward: -2138.1373911224528 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 2.7623

training\03_nedc.mat
SOC is nan...
elapsed_time: 69.264
Episode: 187 Exploration P: 0.0272 Total reward: -2165.0839816021526 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 2.3632

training\03_nedc.mat
SOC is nan...
elapsed_time: 69.055
Episode: 188 Exploration P: 0.0269 Total reward: -2124.491035011917 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 2.6178

training\03_nedc.mat
SOC is nan...
elapsed_time: 69.347
Episode: 189 Exploration P: 0.0265 Total reward: -2104.660886732595 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 2.7719

training\03_nedc.mat
SOC is nan...
elapsed_time: 69.066
Episode: 190 Explo

SOC is nan...
elapsed_time: 68.599
Episode: 229 Exploration P: 0.0171 Total reward: -2148.8544759458027 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 2.3290

training\03_nedc.mat
SOC is nan...
elapsed_time: 69.383
Episode: 230 Exploration P: 0.0169 Total reward: -2176.9138180809096 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 2.2616

training\03_nedc.mat
SOC is nan...
elapsed_time: 68.708
Episode: 231 Exploration P: 0.0168 Total reward: -2159.1300448198426 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 2.2904

training\03_nedc.mat
SOC is nan...
elapsed_time: 68.935
Episode: 232 Exploration P: 0.0166 Total reward: -2198.9974064181097 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 1.9727

training\03_nedc.mat
SOC is nan...
elapsed_time: 69.380
Episode: 233 Exploration P: 0.0165 Total reward: -2130.1423916201547 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 2.5119

training\03_nedc.mat
SOC is nan...
elapsed_time: 69.088
Episode: 234 Ex

SOC is nan...
elapsed_time: 70.476
Episode: 273 Exploration P: 0.0128 Total reward: -2176.9028220520568 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 2.0497

training\03_nedc.mat
SOC is nan...
elapsed_time: 77.086
Episode: 274 Exploration P: 0.0127 Total reward: -2165.5532180750247 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 2.2423

training\03_nedc.mat
SOC is nan...
elapsed_time: 72.524
Episode: 275 Exploration P: 0.0127 Total reward: -2140.7095478823503 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 2.4927

training\03_nedc.mat
SOC is nan...
elapsed_time: 68.288
Episode: 276 Exploration P: 0.0126 Total reward: -2176.5275188632613 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 2.1556

training\03_nedc.mat
SOC is nan...
elapsed_time: 71.904
Episode: 277 Exploration P: 0.0126 Total reward: -2184.2354639418427 SOC: nan Cumulative_SOC_deviation: nan Fuel Consumption: 2.1974

training\03_nedc.mat
SOC is nan...
elapsed_time: 71.014
Episode: 278 Ex

KeyboardInterrupt: 

In [ ]:
with open("DDPG1.pkl", "wb") as f: 
    pickle.dump(results_dict, f, pickle.HIGHEST_PROTOCOL)